# Movie Scrape

In [220]:
# Start
import os
import sys 

import requests

from bs4 import BeautifulSoup

import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

import time

In [221]:
# Initiate Chromedriver and go to URL
chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

url = 'http://www.imdb.com/feature/genre/'

driver = webdriver.Chrome(chromedriver)
driver.get(url)

In [222]:
from bs4 import BeautifulSoup

# soup = BeautifulSoup(driver.page_source, 'html.parser')

In [ ]:
# Scrape Categories and then Scrape Movies from those Categories
import re

pages_scraped = 0
movies = {}
category_urls = []
category_index = 0                             
prefix = 'http://www.imdb.com'
response = requests.get(driver.current_url)
pages_scraped_current_genre = 0
genre_scrapes = str(pages_scraped_current_genre)
export_number = 1
export_no = str(export_number)

csv_columns = []

def scraper():

    print('Working on export ' + export_no)
    
    def scrape_categories():
#         url = 'http://www.imdb.com/feature/genre/'
#         driver.get(url)

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        movie_section = soup.findAll('div', class_='article')[5]
        genre_items = movie_section.findAll('a',href=True)
        genre_names = [item.text.split('(')[0].strip().lower().replace(' ','-') for item in genre_items]

        for name in genre_names:
            category_url = 'http://www.imdb.com/search/title?genres=%s&title_type=feature&explore=genres' %(name)
            category_urls.append(category_url)

        loop_through_categories(category_urls)

        
    def loop_through_categories(category_urls):
#         print('we got to loop_through_categories')
        url = category_urls[category_index]
        driver.get(url)
        get_movie_listings()

    
    def get_movie_listings():
#         print('we got to get_movie_listings')
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        listings = soup.find_all("div",class_='lister-item-content')
        time.sleep(0.1)
        get_movie_data(listings)
 

    def get_movie_data(listings):
#         print('we got to get_movie_data')
        global pages_scraped
        global category_index
        global pages_scraped_current_genre
        global movies
        
        for listing in listings:
            scrape_details(listing)
            
        pages_scraped+=1
#         print(pages_scraped)
        
        if pages_scraped == 100:                  # make 100                         
            export_to_json(movies)
            pages_scraped = 0
            movies = {}
#             return
        
        try:
            driver.find_element_by_class_name("lister-page-next").click()
            pages_scraped_current_genre += 1
        except:
            category_index += 1
            pages_scraped_current_genre = 0
            if category_index <= 23:
                loop_through_categories(category_urls)
        
        if response.status_code != 404:
            time.sleep(0.1)
            get_movie_listings()
        else:
            category_index += 1
            pages_scraped_current_genre = 0
            if category_index <= 23:
                loop_through_categories(category_urls)
            else: 
                export_to_json(movies)
                print("Done")

                
    def scrape_details(listing):
        global movies
#         print('we got to get_movie_data')

        url = listing.find('a')['href']
        url = prefix+url.split("?")[0]
        movies[url] = {}
        
        title = listing.find('a').text 
        movies[url]['Title'] = title
        
        year = listing.find('span', class_='lister-item-year text-muted unbold')
        year = year.text if year else None
        movies[url]['Year'] = re.sub('[^a-za-z0-9]+','', year)
        
        rating = listing.find('div', class_='inline-block ratings-imdb-rating')
        rating = rating.text.split('(')[0].strip() if rating else None
        movies[url]['Rating'] = rating
        
        votes = listing.find('p', class_='sort-num_votes-visible')
        votes = votes.findAll('span') if votes else None
        votes = votes[1].text if votes and len(votes)>1 else None
        movies[url]['Votes'] = votes
        
        metascore = listing.find('div', class_='inline-block ratings-metascore')
        metascore = metascore.text[0:4].split('(')[0].strip() if metascore else None
        movies[url]['Metascore'] = metascore
        
        gross = listing.find('p', class_='sort-num_votes-visible')
        gross = gross.findAll('span') if gross else None
        gross = gross[4] if gross and len(gross)>=5 else None
        gross = gross.text.replace('$','').replace('M','') if gross else None
        movies[url]['Gross'] = gross  
        
        genre = listing.find('span', class_ = 'genre')
        genre = genre.text.split('(')[0].strip() if genre else None
        movies[url]['Genre'] = genre
        
        certificate = listing.find('span', class_ = 'certificate')
        certificate = certificate.text if certificate else None
        movies[url]['Certificate'] = certificate
        
        runtime = listing.find('span', class_ = 'runtime')
        runtime = runtime.text if runtime else None
        movies[url]['Runtime'] = runtime

        people = listing.findAll('p')[2]
        director = people.find('a')
        director = director.text if director else None
        movies[url]['Director'] = director

        stars = people.findAll('a')[1:5]
        i=0
        for star in stars:
            star = stars[i]
            star = star.text if star else None
            movies[url]['Star%s' %str(i+1)] = star
            starURL = stars[i]['href']
            if not starURL:
                return None
            movies[url]['Star%s_URL' %str(i+1)] = prefix+starURL
            i += 1
    

    def export_to_json(movies):
        global export_no
        global export_number
        with open('genre_export_%s.json' %(export_no),'w')  as f:    #renamed from 'export_'
            json.dump(movies,f)
            
        export_number += 1
        export_no = str(export_number)
        print('Working on export ' + export_no)

    scrape_categories()

scraper()

# Importing and Dropping Duplicates

In [1]:
import pandas as pd
import json

In [7]:
imports = []
for i in range(1,41):
    imports.append('genre_exports/export_%s.json' %(str(i)))   #rename to genre_export

In [8]:
imports

['genre_exports/export_1.json',
 'genre_exports/export_2.json',
 'genre_exports/export_3.json',
 'genre_exports/export_4.json',
 'genre_exports/export_5.json',
 'genre_exports/export_6.json',
 'genre_exports/export_7.json',
 'genre_exports/export_8.json',
 'genre_exports/export_9.json',
 'genre_exports/export_10.json',
 'genre_exports/export_11.json',
 'genre_exports/export_12.json',
 'genre_exports/export_13.json',
 'genre_exports/export_14.json',
 'genre_exports/export_15.json',
 'genre_exports/export_16.json',
 'genre_exports/export_17.json',
 'genre_exports/export_18.json',
 'genre_exports/export_19.json',
 'genre_exports/export_20.json',
 'genre_exports/export_21.json',
 'genre_exports/export_22.json',
 'genre_exports/export_23.json',
 'genre_exports/export_24.json',
 'genre_exports/export_25.json',
 'genre_exports/export_26.json',
 'genre_exports/export_27.json',
 'genre_exports/export_28.json',
 'genre_exports/export_29.json',
 'genre_exports/export_30.json',
 'genre_exports/exp

In [9]:
movies_df = pd.DataFrame()

In [10]:
for imp in imports:
    movies_df = movies_df.append(pd.read_json(imp, orient='index'))

# mov = mov.append(pd.read_json('export_2.json', orient='index'))
# mov = mov.append(pd.read_json('export_3.json', orient='index'))

In [11]:
len(movies_df)

188326

### Use New DataFrame

In [12]:
movies = pd.DataFrame(movies_df)

In [13]:
movies['ID'] = movies.index

In [14]:
movies.head()

,Certificate,Director,Genre,Gross,Metascore,Rating,Runtime,Star1,Star1_URL,Star2,Star2_URL,Star3,Star3_URL,Star4,Star4_URL,Title,Votes,Year,ID
http://www.imdb.com/title/tt0006206/,Not Rated,Louis Feuillade,"Action, Adventure, Crime",NaN,NaN,6.9,421 min,Musidora,http://www.imdb.com/name/nm0615736/?ref_=adv_l...,Édouard Mathé,http://www.imdb.com/name/nm0559013/?ref_=adv_l...,Marcel Lévesque,http://www.imdb.com/name/nm0529695/?ref_=adv_l...,Jean Aymé,http://www.imdb.com/name/nm0043923/?ref_=adv_l...,Les vampires,"3,519",1915,http://www.imdb.com/title/tt0006206/
http://www.imdb.com/title/tt0009682/,Not Rated,Scott Sidney,"Action, Adventure",3.27,NaN,5.9,73 min,Elmo Lincoln,http://www.imdb.com/name/nm0511104/?ref_=adv_l...,Enid Markey,http://www.imdb.com/name/nm0548402/?ref_=adv_l...,True Boardman,http://www.imdb.com/name/nm0090204/?ref_=adv_l...,Kathleen Kirkham,http://www.imdb.com/name/nm0456615/?ref_=adv_l...,Tarzan of the Apes,541,1918,http://www.imdb.com/title/tt0009682/
http://www.imdb.com/title/tt0014720/,Passed,James P. Hogan,"Action, Adventure, Drama",NaN,NaN,4.7,60 min,Thunder the Dog,http://www.imdb.com/name/nm1299062/?ref_=adv_l...,Clara Bow,http://www.imdb.com/name/nm0001966/?ref_=adv_l...,Harold Austin,http://www.imdb.com/name/nm0042395/?ref_=adv_l...,Eddie Phillips,http://www.imdb.com/name/nm0680376/?ref_=adv_l...,Black Lightning,9,1924,http://www.imdb.com/title/tt0014720/
http://www.imdb.com/title/tt0015163/,Unrated,Donald Crisp,"Action, Comedy, Romance",0.30,NaN,8.0,59 min,Buster Keaton,http://www.imdb.com/name/nm0000036/?ref_=adv_l...,Buster Keaton,http://www.imdb.com/name/nm0000036/?ref_=adv_l...,Kathryn McGuire,http://www.imdb.com/name/nm0570230/?ref_=adv_l...,Frederick Vroom,http://www.imdb.com/name/nm0089162/?ref_=adv_l...,The Navigator,"7,287",1924,http://www.imdb.com/title/tt0015163/
http://www.imdb.com/title/tt0015324/,Not Rated,Buster Keaton,"Action, Comedy, Romance",NaN,NaN,8.3,45 min,Buster Keaton,http://www.imdb.com/name/nm0000036/?ref_=adv_l...,Kathryn McGuire,http://www.imdb.com/name/nm0570230/?ref_=adv_l...,Joe Keaton,http://www.imdb.com/name/nm0444172/?ref_=adv_l...,Erwin Connelly,http://www.imdb.com/name/nm0175068/?ref_=adv_l...,Sherlock Jr.,"24,694",1924,http://www.imdb.com/title/tt0015324/


In [15]:
movies = movies.drop_duplicates(subset='ID',keep='first')

In [16]:
len(movies)

106906

## Drop NaN Ratings

In [17]:
movies['Rating'].isnull().sum()

29609

In [18]:
movies = movies.dropna(subset = ['Rating'])

In [19]:
len(movies)

77297

## Keep Only Movies with Drop Movies with 1000 Votes or More

In [20]:
movies['Votes'] = movies['Votes'].str.replace(',','')

In [21]:
movies['Votes']

http://www.imdb.com/title/tt0006206/     3519
http://www.imdb.com/title/tt0009682/      541
http://www.imdb.com/title/tt0014720/        9
http://www.imdb.com/title/tt0015163/     7287
http://www.imdb.com/title/tt0015324/    24694
http://www.imdb.com/title/tt0017925/    61402
http://www.imdb.com/title/tt0018578/     8998
http://www.imdb.com/title/tt0019421/    10630
http://www.imdb.com/title/tt0019532/      269
http://www.imdb.com/title/tt0020815/     1325
http://www.imdb.com/title/tt0021079/     9804
http://www.imdb.com/title/tt0022753/      728
http://www.imdb.com/title/tt0023427/    21444
http://www.imdb.com/title/tt0023551/     5944
http://www.imdb.com/title/tt0025004/     1958
http://www.imdb.com/title/tt0025862/     4124
http://www.imdb.com/title/tt0026164/     1433
http://www.imdb.com/title/tt0026174/    10768
http://www.imdb.com/title/tt0027438/     3741
http://www.imdb.com/title/tt0027623/     1745
http://www.imdb.com/title/tt0029047/     1707
http://www.imdb.com/title/tt002908

In [22]:
movies['Votes'] = movies['Votes'].astype(int)

In [23]:
len(movies[movies['Votes'] >= 500])

29717

In [24]:
movies = movies[movies['Votes'] >= 500]

In [30]:
movies.loc[movies['Star1'] == 'Cary Murnion']

,Certificate,Director,Genre,Gross,Metascore,Rating,Runtime,Star1,Star1_URL,Star2,Star2_URL,Star3,Star3_URL,Star4,Star4_URL,Title,Votes,Year,ID
http://www.imdb.com/title/tt2490326/,R,Jonathan Milott,"Comedy, Horror, Sci-Fi",0.06,49.0,5.6,88 min,Cary Murnion,http://www.imdb.com/name/nm5722382/?ref_=adv_l...,Elijah Wood,http://www.imdb.com/name/nm0000704/?ref_=adv_l...,Rainn Wilson,http://www.imdb.com/name/nm0933988/?ref_=adv_l...,Alison Pill,http://www.imdb.com/name/nm0683467/?ref_=adv_l...,Cooties,18376,2014,http://www.imdb.com/title/tt2490326/


# Scrape Details with Movie URL

In [596]:
len(movies.index)

29717

In [597]:
movies.index

Index(['http://www.imdb.com/title/tt0006206/',
       'http://www.imdb.com/title/tt0009682/',
       'http://www.imdb.com/title/tt0015163/',
       'http://www.imdb.com/title/tt0015324/',
       'http://www.imdb.com/title/tt0017925/',
       'http://www.imdb.com/title/tt0018578/',
       'http://www.imdb.com/title/tt0019421/',
       'http://www.imdb.com/title/tt0020815/',
       'http://www.imdb.com/title/tt0021079/',
       'http://www.imdb.com/title/tt0022753/',
       ...
       'http://www.imdb.com/title/tt0439504/',
       'http://www.imdb.com/title/tt0457530/',
       'http://www.imdb.com/title/tt0495747/',
       'http://www.imdb.com/title/tt1384925/',
       'http://www.imdb.com/title/tt1388402/',
       'http://www.imdb.com/title/tt1555110/',
       'http://www.imdb.com/title/tt2014202/',
       'http://www.imdb.com/title/tt2140381/',
       'http://www.imdb.com/title/tt4893452/',
       'http://www.imdb.com/title/tt5143890/'],
      dtype='object', length=29717)

In [598]:
movie_urls = list(movies.index)

In [599]:
len(movie_urls)

29717

In [602]:
movie_urls[2]

'http://www.imdb.com/title/tt7711342/'

In [600]:
movie_urls = sorted(movie_urls,reverse = True)

In [601]:
movie_urls[-200:]

['http://www.imdb.com/title/tt0020018/',
 'http://www.imdb.com/title/tt0019993/',
 'http://www.imdb.com/title/tt0019959/',
 'http://www.imdb.com/title/tt0019946/',
 'http://www.imdb.com/title/tt0019933/',
 'http://www.imdb.com/title/tt0019901/',
 'http://www.imdb.com/title/tt0019824/',
 'http://www.imdb.com/title/tt0019823/',
 'http://www.imdb.com/title/tt0019811/',
 'http://www.imdb.com/title/tt0019788/',
 'http://www.imdb.com/title/tt0019777/',
 'http://www.imdb.com/title/tt0019735/',
 'http://www.imdb.com/title/tt0019729/',
 'http://www.imdb.com/title/tt0019702/',
 'http://www.imdb.com/title/tt0019649/',
 'http://www.imdb.com/title/tt0019646/',
 'http://www.imdb.com/title/tt0019644/',
 'http://www.imdb.com/title/tt0019630/',
 'http://www.imdb.com/title/tt0019611/',
 'http://www.imdb.com/title/tt0019585/',
 'http://www.imdb.com/title/tt0019563/',
 'http://www.imdb.com/title/tt0019421/',
 'http://www.imdb.com/title/tt0019415/',
 'http://www.imdb.com/title/tt0019412/',
 'http://www.imd

In [604]:
movies.columns

Index(['Certificate', 'Director', 'Genre', 'Gross', 'Metascore', 'Rating',
       'Runtime', 'Star1', 'Star1_URL', 'Star2', 'Star2_URL', 'Star3',
       'Star3_URL', 'Star4', 'Star4_URL', 'Title', 'Votes', 'Year', 'ID'],
      dtype='object')

In [605]:
movies.loc['http://www.imdb.com/title/tt0102926/']

Certificate                                                    R
Director                                          Jonathan Demme
Genre                                     Crime, Drama, Thriller
Gross                                                     130.74
Metascore                                                     85
Rating                                                       8.6
Runtime                                                  118 min
Star1                                               Jodie Foster
Star1_URL      http://www.imdb.com/name/nm0000149/?ref_=adv_l...
Star2                                            Anthony Hopkins
Star2_URL      http://www.imdb.com/name/nm0000164/?ref_=adv_l...
Star3                                         Lawrence A. Bonney
Star3_URL      http://www.imdb.com/name/nm0095029/?ref_=adv_l...
Star4                                               Kasi Lemmons
Star4_URL      http://www.imdb.com/name/nm0501435/?ref_=adv_l...
Title                    

In [446]:
# url = 'http://www.imdb.com/title/tt6118762/'
# driver.get(url)

country = title_details.find(text=re.compile('Country:')) if title_details else None
country = country.parent.parent.find('a').text if country else None
# .text if country else None
# country = country.text if country else None
country

'UK'

In [606]:
# Initiate Chromedriver and go to URL
# # chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
# os.environ["webdriver.chrome.driver"] = chromedriver

import requests 

# url = 'http://www.imdb.com/'

# url = movie_urls[2532]

# driver = webdriver.Chrome(chromedriver)
# driver.get(url)


In [618]:
# Movie Details
import re
import requests
from bs4 import BeautifulSoup
from datetime import datetime

movies_scraped = 0
movie_details = {}
movie_index = 26477                         
prefix = 'http://www.imdb.com'
export_number = 28
export_no = str(export_number)
load_attempts = 0

def movie_scraper():
    global movie_index
    
    print(str(datetime.now()),': working on export ' + export_no)
    
    for url in movie_urls[26477:]:
        if (movie_index+1) % 20 == 0:
            time.sleep(1.5)
            print('%d: %s' %(movie_index+1, url))
        page = request_page(url)
        get_movie_details(url, page)
        movie_index += 1
    
    export_to_json(movie_details)
    print("Done")
    
def request_page(url):
    global load_attempts
    response = requests.get(url)
    
    if response.status_code == 200:
        return(response.text)
        load_attempts = 0
    else:
        print(response.status_code,'for %s' %(url))
        if response.status_code != 404:
            if load_attempts == 10:
                time.sleep(60)
            else:
                time.sleep(10)
            request_page(url)
            load_attempts += 1

def get_movie_details(url, page):
    soup = BeautifulSoup(page, 'html.parser')

    movie_details[url] = {}

    directorURL = soup.find('div', class_='credit_summary_item')
    directorURL = directorURL.find('a') if directorURL else None
    director = directorURL.find('span') if directorURL else None
    director = director.text if director else None
    directorURL = directorURL['href'] if directorURL else None
    directorURL = prefix+directorURL.split("?")[0] if directorURL else None
    movie_details[url]['Director'] = director
    movie_details[url]['Director_URL'] = directorURL

    title = soup.find('div', class_='title_wrapper').find('h1',itemprop='name')
    title = title.contents[0].replace(u'\xa0', u'')
    movie_details[url]['Title'] = title

    release_date = soup.find('div', class_='title_wrapper')
    release_date = release_date.findAll('a') if release_date else None
    release_date = release_date[-1] if release_date else None
    release_date = release_date.find('meta') if release_date else None
    release_date = str(release_date) if release_date else None
    release_date = re.sub('[^0-9-]+','', release_date) if release_date else None
    movie_details[url]['Release Date'] = release_date

    title_details = soup.find(id='titleDetails')

    country = title_details.find(text=re.compile('Country:')) if title_details else None
    country = country.parent.parent if country else None
    country = country.find('a') if country else None
    country = country.text if country else None
    movie_details[url]['Country'] = country

    language = title_details.find(text=re.compile('Language:')) if title_details else None
    language = language.parent.parent if language else None
    language = language.find('a') if language else None
    language = language.text if language else None
    movie_details[url]['Language'] = language

    primary_filming_location = title_details.find(text=re.compile('Filming Locations:')) if title_details else None
    primary_filming_location = primary_filming_location.parent.parent if primary_filming_location else None
    primary_filming_location = primary_filming_location.find('a') if primary_filming_location else None
    primary_filming_location= primary_filming_location.text if primary_filming_location else None
    movie_details[url]['Primary Filming Location'] = primary_filming_location

    budget = title_details.find(text=re.compile('Budget:')) if title_details else None
    budget = budget.parent.parent if budget else None
    budget = budget.text.strip() if budget else None
    budget = re.sub('[^0-9]+','', budget) if budget else None
    movie_details[url]['Budget'] = budget

    opening_weekend_usa = title_details.find(text=re.compile('Opening Weekend USA:')) if title_details else None
    opening_weekend_usa = opening_weekend_usa.parent.parent if opening_weekend_usa else None
    opening_weekend_usa = opening_weekend_usa.text.strip() if opening_weekend_usa else None
    opening_weekend_usa = re.sub('[^0-9]+','', opening_weekend_usa) if opening_weekend_usa else None
    movie_details[url]['Opening Weekend USA'] = opening_weekend_usa

    #rename gross to gross USA in listing scrape
    global_gross = title_details.find(text=re.compile('Cumulative Worldwide Gross:')) if title_details else None
    global_gross = global_gross.parent.parent if global_gross else None
    global_gross = global_gross.text.strip() if global_gross else None
    global_gross = re.sub('[^0-9]+','', global_gross) if global_gross else None
    movie_details[url]['Cumulative Worldwide Gross'] = global_gross

    gross_usa = title_details.find(text=re.compile('Gross USA:')) if title_details else None
    gross_usa = gross_usa.parent.parent if gross_usa else None
    gross_usa = gross_usa.text.strip() if gross_usa else None
    gross_usa = re.sub('[^0-9]+','', gross_usa) if gross_usa else None
    movie_details[url]['Gross USA'] = gross_usa

    go_to_next_and_export()

def go_to_next_and_export():
    global movies_scraped
    global movie_details
    
    movies_scraped+=1

    if len(movie_details) == 1000:                                         
        export_to_json(movie_details)
        movies_scraped = 0
        movie_details = {}

def export_to_json(movie_details):
    global export_no
    global export_number
    with open('movie_details_export_%s.json' %(export_no),'w')  as f:
        json.dump(movie_details,f)

    export_number += 1
    export_no = str(export_number)
    print(str(datetime.now()),': working on export ' + export_no)

movie_scraper()

2018-01-28 11:15:53.393698 : working on export 28
26480: http://www.imdb.com/title/tt0053434/
26500: http://www.imdb.com/title/tt0053258/
26520: http://www.imdb.com/title/tt0053143/
26540: http://www.imdb.com/title/tt0053033/
26560: http://www.imdb.com/title/tt0052905/
26580: http://www.imdb.com/title/tt0052809/
26600: http://www.imdb.com/title/tt0052676/
26620: http://www.imdb.com/title/tt0052563/
26640: http://www.imdb.com/title/tt0052311/
26660: http://www.imdb.com/title/tt0052190/
26680: http://www.imdb.com/title/tt0052002/
26700: http://www.imdb.com/title/tt0051818/
26720: http://www.imdb.com/title/tt0051713/
26740: http://www.imdb.com/title/tt0051565/
26760: http://www.imdb.com/title/tt0051432/
26780: http://www.imdb.com/title/tt0051198/
26800: http://www.imdb.com/title/tt0051020/
26820: http://www.imdb.com/title/tt0050870/
26840: http://www.imdb.com/title/tt0050749/
26860: http://www.imdb.com/title/tt0050613/
26880: http://www.imdb.com/title/tt0050470/
26900: http://www.imdb.com

len(movie)

movies_scraped

with open('movie_details_export_%s.json' %(export_no),'w')  as f:
        json.dump(movie_details,f)

movies_scraped = 0
movie_details = {}

sys.getrecursionlimit()

movie_index

movie_urls[1331]

len(movie_details)

movie_details[movie_urls[movie_index+10]] = {}

movie_details['http://www.imdb.com/title/tt0000335/']['movie'] = 'new title'

## Merge/join based on movie url

# Scrape Director and Stars URLs

## Create Dict for Director

## Create Dict for Stars

# Misc / Testing

## .json: Exporting, Importing, and Appending

In [84]:
testing = {'Case 1' : {'header1' : 'value1', 'header2': 'value2', 'header3': 'value3'}, 
           'Case 2' : {'header1' : 'value1', 'header2': 'value2', 'header3': 'value3'},
           'Case 3' : {'header1' : 'value1', 'header2': 'value2', 'header3': 'value3'},
           'Case 4' : {'header1' : 'value1', 'header2': 'value2', 'header3': 'value3'},
           'Case 5' : {'header1' : 'value1', 'header2': 'value2', 'header3': 'value3'},
           'Case 6' : {'header1' : 'value1', 'header2': 'value2', 'header3': 'value3'},
           'Case 7' : {'header1' : 'value1', 'header2': 'value2', 'header3': 'value3'},
           'Case 8' : {'header1' : 'value1', 'header2': 'value2', 'header3': 'value3'},
           'Case 9' : {'header1' : 'value1', 'header2': 'value2', 'header3': 'value3'},
           'Case 10' : {'header1' : 'value1', 'header2': 'value2', 'header3': 'value3'},
          }

In [85]:
export_number = 5
export_no = str(export_number)

In [87]:
with open('my.json','w')  as f:
    json.dump(testing,f)

In [88]:
import pandas as pd
pd.read_json('my.json',orient='index')

,header1,header2,header3
Case 1,value1,value2,value3
Case 10,value1,value2,value3
Case 2,value1,value2,value3
Case 3,value1,value2,value3
Case 4,value1,value2,value3
Case 5,value1,value2,value3
Case 6,value1,value2,value3
Case 7,value1,value2,value3
Case 8,value1,value2,value3
Case 9,value1,value2,value3


In [100]:
with open('moviestest.json','w')  as f:
    json.dump(movies,f)

In [ ]:
import pandas as pd
directors_df = pd.read_json('director_details_export_11.json',orient='index')

In [ ]:
directors_df.head()

In [128]:
len(movies_df)

498

In [140]:
movies_df = movies_df.append(pd.read_json('export_2.json',orient='index'))

In [141]:
movies_df2

,Certificate,Director,Genre,Gross,Metascore,Movie_URL,Rating,Runtime,Star1,Star1_URL,Star2,Star2_URL,Star3,Star3_URL,Star4,Star4_URL,Votes,Year
'C'-Man,None,Joseph Lerner,"Crime, Drama, Film-Noir",NaN,NaN,http://www.imdb.com/title/tt0041218/,6.1,75 min,Dean Jagger,http://www.imdb.com/name/nm0415591/?ref_=adv_l...,John Carradine,http://www.imdb.com/name/nm0001017/?ref_=adv_l...,Harry Landers,http://www.imdb.com/name/nm0484636/?ref_=adv_l...,Lottie Elwen,http://www.imdb.com/name/nm0255886/?ref_=adv_l...,NaN,1949
"10,000 BC",PG-13,Roland Emmerich,"Action, Adventure, Drama",NaN,34.0,http://www.imdb.com/title/tt0443649/,5.1,109 min,Camilla Belle,http://www.imdb.com/name/nm0004741/?ref_=adv_l...,Steven Strait,http://www.imdb.com/name/nm1711829/?ref_=adv_l...,Marco Khan,http://www.imdb.com/name/nm0434879/?ref_=adv_l...,Cliff Curtis,http://www.imdb.com/name/nm0193295/?ref_=adv_l...,NaN,2008
12 Strong,R,Nicolai Fuglsig,"Action, Drama, History",NaN,55.0,http://www.imdb.com/title/tt1413492/,7.1,130 min,Chris Hemsworth,http://www.imdb.com/name/nm1165110/?ref_=adv_l...,Michael Shannon,http://www.imdb.com/name/nm0788335/?ref_=adv_l...,Michael Peña,http://www.imdb.com/name/nm0671567/?ref_=adv_l...,Navid Negahban,http://www.imdb.com/name/nm1041023/?ref_=adv_l...,NaN,2018
12 Years a Slave,R,Steve McQueen,"Biography, Drama, History",NaN,96.0,http://www.imdb.com/title/tt2024544/,8.1,134 min,Chiwetel Ejiofor,http://www.imdb.com/name/nm0252230/?ref_=adv_l...,Michael Kenneth Williams,http://www.imdb.com/name/nm0931324/?ref_=adv_l...,Michael Fassbender,http://www.imdb.com/name/nm1055413/?ref_=adv_l...,Brad Pitt,http://www.imdb.com/name/nm0000093/?ref_=adv_l...,NaN,2013
13 Hours,R,Michael Bay,"Action, Drama, History",NaN,48.0,http://www.imdb.com/title/tt4172430/,7.3,144 min,John Krasinski,http://www.imdb.com/name/nm1024677/?ref_=adv_l...,Pablo Schreiber,http://www.imdb.com/name/nm1032567/?ref_=adv_l...,James Badge Dale,http://www.imdb.com/name/nm0197647/?ref_=adv_l...,David Denman,http://www.imdb.com/name/nm0219292/?ref_=adv_l...,NaN,2016
5 Steps to Danger,Approved,Henry S. Kesler,"Crime, Drama, Film-Noir",NaN,NaN,http://www.imdb.com/title/tt0050087/,6.2,81 min,Ruth Roman,http://www.imdb.com/name/nm0738746/?ref_=adv_l...,Sterling Hayden,http://www.imdb.com/name/nm0001330/?ref_=adv_l...,Werner Klemperer,http://www.imdb.com/name/nm0459252/?ref_=adv_l...,Richard Gaines,http://www.imdb.com/name/nm0301348/?ref_=adv_l...,NaN,1957
6 Days,R,Toa Fraser,"Action, Drama, History",NaN,36.0,http://www.imdb.com/title/tt4703048/,6.2,94 min,Jamie Bell,http://www.imdb.com/name/nm0068260/?ref_=adv_l...,Mark Strong,http://www.imdb.com/name/nm0835016/?ref_=adv_l...,Abbie Cornish,http://www.imdb.com/name/nm0180411/?ref_=adv_l...,Martin Shaw,http://www.imdb.com/name/nm0789864/?ref_=adv_l...,NaN,2017
A Bullet Is Waiting,Approved,John Farrow,"Crime, Drama, Film-Noir",NaN,NaN,http://www.imdb.com/title/tt0046812/,5.6,90 min,Rory Calhoun,http://www.imdb.com/name/nm0001983/?ref_=adv_l...,Jean Simmons,http://www.imdb.com/name/nm0001739/?ref_=adv_l...,Stephen McNally,http://www.imdb.com/name/nm0573640/?ref_=adv_l...,Brian Aherne,http://www.imdb.com/name/nm0000731/?ref_=adv_l...,NaN,1954
A Dead Man Among the Living,None,Borivoj Zeman,"Crime, Drama, Film-Noir",NaN,NaN,http://www.imdb.com/title/tt0194183/,6.7,90 min,Karel Höger,http://www.imdb.com/name/nm0405859/?ref_=adv_l...,Eduard Dubský,http://www.imdb.com/name/nm0239627/?ref_=adv_l...,Vjaceslav Irmanov,http://www.imdb.com/name/nm0410041/?ref_=adv_l...,Zdenka Procházková,http://www.imdb.com/name/nm0698324/?ref_=adv_l...,NaN,1947
A Prize of Gold,None,Mark Robson,"Adventure, Crime, Drama",NaN,NaN,http://www.imdb.com/title/tt0048514/,6.3,98 min,Richard Widmark,http://www.imdb.com/name/nm0001847/?ref_=adv_l...,Mai Zetterling,http://www.imdb.com/name/nm0955195/?ref_=adv_l...,Nigel Patrick,http://www.imdb.com/name/nm0665902/?ref_=adv_l...,George Cole,http://www.imdb.com/name/nm0170552/?ref_=adv_l...,NaN,1955


In [134]:
type(new)

pandas.core.frame.DataFrame

In [146]:
movies_total = pd.concat([movies_df, movies_df2])

In [147]:
len(movies_total)

953

In [119]:
movies_df.tail()

,Certificate,Director,Genre,Gross,Metascore,Movie_URL,Rating,Runtime,Star1,Star1_URL,Star2,Star2_URL,Star3,Star3_URL,Star4,Star4_URL,Votes,Year
World in My Corner,Approved,Jesse Hibbs,"Drama, Film-Noir, Sport",NaN,NaN,http://www.imdb.com/title/tt0049965/,7.4,82 min,Audie Murphy,http://www.imdb.com/name/nm0001559/?ref_=adv_l...,Barbara Rush,http://www.imdb.com/name/nm0750640/?ref_=adv_l...,Jeff Morrow,http://www.imdb.com/name/nm0607504/?ref_=adv_l...,John McIntire,http://www.imdb.com/name/nm0570615/?ref_=adv_l...,NaN,1956
You Can't Escape,None,Wilfred Eades,"Film-Noir, Thriller",NaN,NaN,http://www.imdb.com/title/tt0221722/,6.6,77 min,Noelle Middleton,http://www.imdb.com/name/nm0585548/?ref_=adv_l...,Guy Rolfe,http://www.imdb.com/name/nm0738212/?ref_=adv_l...,Robert Urquhart,http://www.imdb.com/name/nm0882048/?ref_=adv_l...,Peter Reynolds,http://www.imdb.com/name/nm0721906/?ref_=adv_l...,NaN,1957
Young and Wild,Approved,William Witney,"Crime, Drama, Film-Noir",NaN,NaN,http://www.imdb.com/title/tt0052416/,6.1,69 min,Gene Evans,http://www.imdb.com/name/nm0262775/?ref_=adv_l...,Scott Marlowe,http://www.imdb.com/name/nm0549325/?ref_=adv_l...,Carolyn Kearney,http://www.imdb.com/name/nm0443920/?ref_=adv_l...,Robert Arthur,http://www.imdb.com/name/nm0037816/?ref_=adv_l...,NaN,1958
Zodiac,R,David Fincher,"Crime, Drama, History",NaN,78.0,http://www.imdb.com/title/tt0443706/,7.7,157 min,Jake Gyllenhaal,http://www.imdb.com/name/nm0350453/?ref_=adv_l...,Robert Downey Jr.,http://www.imdb.com/name/nm0000375/?ref_=adv_l...,Mark Ruffalo,http://www.imdb.com/name/nm0749263/?ref_=adv_l...,Anthony Edwards,http://www.imdb.com/name/nm0000381/?ref_=adv_l...,NaN,2007
Ça va barder,None,John Berry,"Action, Adventure, Crime",NaN,NaN,http://www.imdb.com/title/tt0047697/,6.2,90 min,Eddie Constantine,http://www.imdb.com/name/nm0176061/?ref_=adv_l...,May Britt,http://www.imdb.com/name/nm0110067/?ref_=adv_l...,Jean Danet,http://www.imdb.com/name/nm0199431/?ref_=adv_l...,Lyla Rocco,http://www.imdb.com/name/nm0733689/?ref_=adv_l...,NaN,1955


In [79]:
movies

{"'G' Men": {'Certificate': 'Approved',
  'Director': 'William Keighley',
  'Genre': 'Crime, Drama, Film-Noir',
  'Gross': None,
  'Metascore': None,
  'Movie_URL': 'http://www.imdb.com/title/tt0026393/',
  'Rating': '7.2',
  'Runtime': '85 min',
  'Star1': 'James Cagney',
  'Star1_URL': 'http://www.imdb.com/name/nm0000010/?ref_=adv_li_st_0',
  'Star2': 'Margaret Lindsay',
  'Star2_URL': 'http://www.imdb.com/name/nm0512267/?ref_=adv_li_st_1',
  'Star3': 'Ann Dvorak',
  'Star3_URL': 'http://www.imdb.com/name/nm0245304/?ref_=adv_li_st_2',
  'Star4': 'Robert Armstrong',
  'Star4_URL': 'http://www.imdb.com/name/nm0035877/?ref_=adv_li_st_3',
  'Votes': None,
  'Year': '1935'},
 '13 Rue Madeleine': {'Certificate': 'Approved',
  'Director': 'Henry Hathaway',
  'Genre': 'Action, Adventure, Drama',
  'Gross': None,
  'Metascore': None,
  'Movie_URL': 'http://www.imdb.com/title/tt0038279/',
  'Rating': '7.0',
  'Runtime': '95 min',
  'Star1': 'James Cagney',
  'Star1_URL': 'http://www.imdb.com/n

In [39]:
for movie in testing.keys():
    for i in testing[movie]:
        print(i, testing[movie][i])

header1 value1
header2 value2
header3 value3
header1 value1
header2 value2
header3 value3
header1 value1
header2 value2
header3 value3
header1 value1
header2 value2
header3 value3
header1 value1
header2 value2
header3 value3
header1 value1
header2 value2
header3 value3
header1 value1
header2 value2
header3 value3
header1 value1
header2 value2
header3 value3
header1 value1
header2 value2
header3 value3
header1 value1
header2 value2
header3 value3


In [29]:
testing['Case 1']

{'header1': 'value1', 'header2': 'value2', 'header3': 'value3'}

In [40]:
testing.keys()

dict_keys(['Case 1', 'Case 2', 'Case 3', 'Case 4', 'Case 5', 'Case 6', 'Case 7', 'Case 8', 'Case 9', 'Case 10'])

### Test

In [232]:
listings = soup.find_all("div",class_='lister-item-content')

In [233]:
test = []

In [234]:
for listing in listings:
    test.append(listing.find('a').text)

In [235]:
test

['Jumanji: Welcome to the Jungle',
 'Star Wars: The Last Jedi',
 'The Commuter',
 'Avengers: Infinity War',
 '12 Strong',
 'Black Panther',
 'Bright',
 'Den of Thieves',
 'Proud Mary',
 'Dunkirk',
 'Justice League',
 'True Lies',
 'Tomb Raider',
 'Solo: A Star Wars Story',
 'Venom',
 'X-Men: Dark Phoenix',
 'Maze Runner: The Death Cure',
 'Kingsman: The Golden Circle',
 'Acts of Violence',
 'Geostorm',
 'Pirates of the Caribbean: Dead Men Tell No Tales',
 'The Foreigner',
 'American Made',
 'Thor: Ragnarok',
 'Guardians of the Galaxy Vol. 2',
 'M:I 6 - Mission Impossible',
 'The Fate of the Furious',
 'Baby Driver',
 'Beast of Burden',
 'Logan',
 'Ant-Man and the Wasp',
 'The Mountain Between Us',
 'Ready Player One',
 'Spider-Man: Homecoming',
 'The Predator',
 'Wonder Woman',
 'Untitled Deadpool Sequel',
 'King Arthur: Legend of the Sword',
 'Aquaman',
 'Atomic Blonde',
 'Valerian and the City of a Thousand Planets',
 'The Dark Knight',
 "The Hitman's Bodyguard",
 'What Happened to M

In [101]:
movies[title]

{'Certificate': 'PG-13',
 'Genre': 'Action, Adventure, Comedy',
 'Runtime': '119 min',
 'URL': 'http://www.imdb.com/title/tt2283362/?ref_=adv_li_tt',
 'Year': '2017'}

In [147]:
people

<p class="">
    Director:
<a href="/name/nm0440458/?ref_=adv_li_dr_0">Jake Kasdan</a>
<span class="ghost">|</span> 
    Stars:
<a href="/name/nm0425005/?ref_=adv_li_st_0">Dwayne Johnson</a>, 
<a href="/name/nm2394794/?ref_=adv_li_st_1">Karen Gillan</a>, 
<a href="/name/nm0366389/?ref_=adv_li_st_2">Kevin Hart</a>, 
<a href="/name/nm0085312/?ref_=adv_li_st_3">Jack Black</a>
</p>

In [153]:
stars

[<a href="/name/nm0425005/?ref_=adv_li_st_0">Dwayne Johnson</a>,
 <a href="/name/nm2394794/?ref_=adv_li_st_1">Karen Gillan</a>,
 <a href="/name/nm0366389/?ref_=adv_li_st_2">Kevin Hart</a>,
 <a href="/name/nm0085312/?ref_=adv_li_st_3">Jack Black</a>]

In [164]:
'hoi'+str(5)

'hoi5'

In [242]:
# # Function to create dictionary in dictionary for each listing
# # and go through all the data to include in dictionary for each listing
# import re

# movies = {}

# prefix = 'http://www.imdb.com'
# listings = soup.find_all("div",class_='lister-item-content')
# # listings[0].a.get_text

# def get_movie_data():
#     for listing in listings:

#         title = listing.find('a').text 
#         movies[title] = {}

#         url = listing.find('a')['href']
#         movies[title]['Movie_URL']=prefix+url.split("?")[0]

#         year = listing.find('span', class_='lister-item-year text-muted unbold').text
#         if not year:
#             return None 
#         movies[title]['Year'] = re.sub('[^a-za-z0-9]+','', year)

#         certificate = listing.find('span', class_ = 'certificate').text
#         if not certificate:
#             return None
#         movies[title]['Certificate'] = certificate

#         runtime = listing.find('span', class_ = 'runtime').text
#         if not runtime:
#             return None
#         movies[title]['Runtime'] = runtime

#         genre = listing.find('span', class_ = 'genre').text.split('(')[0].strip()
#         if not genre:
#             return None
#         movies[title]['Genre'] = genre

#         rating = listing.find('div', class_='inline-block ratings-imdb-rating').text.split('(')[0].strip()
#         if not rating:
#             return None

#         movies[title]['Rating'] = rating

#         metascore = listing.find('div', class_='inline-block ratings-metascore').text[0:4].split('(')[0].strip()
#         movies[title]['Metascore'] = metascore

#         people = listing.findAll('p')[2]
#         director = people.find('a').text
#         if not director:
#             return None
#         movies[title]['Director'] = director

#         stars = people.findAll('a')[1:5]
#         i=0
#         for star in stars:
#             star = stars[i].text
#             if not star:
#                 return None
#             movies[title]['Star%s' %str(i+1)] = star
#             starURL = stars[i]['href']
#             if not starURL:
#                 return None
#             movies[title]['Star%s_URL' %str(i+1)] = prefix+starURL
#             i += 1

#         votes = listing.find('p', class_='sort-num_votes-visible').findAll('span')[1].text
#         if not votes:
#             return None
#         movies[title]['Votes'] = votes

#         gross = listing.find('p', class_='sort-num_votes-visible').findAll('span')[4].text.replace('$','').replace('M','')
#         if not gross:
#             return None
#         movies[title]['Gross'] = gross

# # What to return if it can't find it? Nan, but how?   
# # "NoneType' object has no attribute 'text'"
# get_movie_data()

AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
# # Function to create dictionary in dictionary for each listing
# # and go through all the data to include in dictionary for each listing
# import re

# movies = {}

# prefix = 'http://www.imdb.com'
# listings = soup.find_all("div",class_='lister-item-content')
# # listings[0].a.get_text

# def get_movie_data():
#     for listing in listings:

#         title = listing.find('a').text 
#         movies[title] = {}

#         url = listing.find('a')['href']
#         movies[title]['Movie_URL']=prefix+url.split("?")[0]

#         year = listing.find('span', class_='lister-item-year text-muted unbold').text
#         if not year:
#             return None 
#         movies[title]['Year'] = re.sub('[^a-za-z0-9]+','', year)

#         certificate = listing.find('span', class_ = 'certificate')
#         if not certificate:
#             return None
#         else:
#             certificate = certificate.text
#             return certificate
#         movies[title]['Certificate'] = certificate

#         runtime = listing.find('span', class_ = 'runtime').text
#         if not runtime:
#             return None
#         movies[title]['Runtime'] = runtime

#         genre = listing.find('span', class_ = 'genre').text.split('(')[0].strip()
#         if not genre:
#             return None
#         movies[title]['Genre'] = genre

#         rating = listing.find('div', class_='inline-block ratings-imdb-rating').text.split('(')[0].strip()
#         if not rating:
#             return None

#         movies[title]['Rating'] = rating

#         metascore = listing.find('div', class_='inline-block ratings-metascore').text[0:4].split('(')[0].strip()
#         movies[title]['Metascore'] = metascore

#         people = listing.findAll('p')[2]
#         director = people.find('a').text
#         if not director:
#             return None
#         movies[title]['Director'] = director

#         stars = people.findAll('a')[1:5]
#         i=0
#         for star in stars:
#             star = stars[i].text
#             if not star:
#                 return None
#             movies[title]['Star%s' %str(i+1)] = star
#             starURL = stars[i]['href']
#             if not starURL:
#                 return None
#             movies[title]['Star%s_URL' %str(i+1)] = prefix+starURL
#             i += 1

#         votes = listing.find('p', class_='sort-num_votes-visible').findAll('span')[1].text
#         if not votes:
#             return None
#         movies[title]['Votes'] = votes

#         gross = listing.find('p', class_='sort-num_votes-visible').findAll('span')[4].text.replace('$','').replace('M','')
#         if not gross:
#             return None
#         movies[title]['Gross'] = gross

# # What to return if it can't find it? Nan, but how?   
# # "NoneType' object has no attribute 'text'"
# get_movie_data()

## Requests to open Genres Page

In [ ]:
# scrape_categories()

In [ ]:
##### Misc
    

#     genre_items = soup.Find('div', class_='desc')
#     for genre in genre_links
#     # Click into Action genre
#     # find the tags for the different categories to scrape and then have the formula iterate through it
#     # Click into Action genre
#     genre_selector = '//a[contains(@href, "search/title?genres=action&title_type=feature&explore=genres")]'
#     driver.find_element_by_xpath(genre_selector).click()

    ##### Question: can you replace the 'action' with a % so that you can fill that from a list of genres?

# At some point save to .csv
## Advice: scrape x number of pages, and then export to .csv, call file 'page 1-x'
## Remove data from memory and continue scraping
## That way, you can already import it to a dataframe and start modeling
# At some point include pause

In [505]:
# Initiate Chromedriver and go to URL
chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

url = 'http://www.imdb.com/'

# url = movie_urls[2532]

driver = webdriver.Chrome(chromedriver)
driver.get(url)

In [506]:
from bs4 import BeautifulSoup

# soup = BeautifulSoup(driver.page_source, 'html.parser')

In [508]:
# Movie Details
import re

movies_scraped = 0
movie_details = {}
movie_index = 0                            
prefix = 'http://www.imdb.com'
response = requests.get(driver.current_url)
export_number = 1
export_no = str(export_number)

def movie_scraper():

    print('Working on export ' + export_no)
    
    def loop_through_movies(movie_urls):
        global movie_index
        global url
    
        url = movie_urls[movie_index]
        driver.get(url)
        
        if response.status_code != 404:
#             time.sleep(0.01)
            get_movie_details()
        else:
            movie_index += 1
            if movie_index <= 106905:
                loop_through_movies(movie_urls)

                

    def get_movie_details():
        soup = BeautifulSoup(driver.page_source, 'html.parser')
    
        global movies_scraped
        global movie_index
        global movie_details
        
        movie_details[url] = {}
        
        directorURL = soup.find('div', class_='credit_summary_item')
        directorURL = directorURL.find('a') if directorURL else None
        director = directorURL.find('span') if directorURL else None
        director = director.text if director else None
        directorURL = directorURL['href'] if directorURL else None
        directorURL = prefix+directorURL.split("?")[0] if directorURL else None
        movie_details[url]['Director'] = director
        movie_details[url]['Director_URL'] = directorURL
        
        title = soup.find('div', class_='title_wrapper').find('h1',itemprop='name')
        title = title.contents[0].replace(u'\xa0', u'')
        movie_details[url]['Title'] = title
        
        release_date = soup.find('div', class_='title_wrapper')
        release_date = release_date.findAll('a')[-1].find('meta') if release_date else None
        release_date = str(release_date) if release_date else None
        release_date = re.sub('[^0-9-]+','', release_date) if release_date else None
        movie_details[url]['Release Date'] = release_date
 
        title_details = soup.find(id='titleDetails')
    
        country = title_details.find(text=re.compile('Country:')) if title_details else None
        country = country.parent.parent.find('a') if country else None
        country = country.text if country else None
        movie_details[url]['Country'] = country
        
        language = title_details.find(text=re.compile('Language:')) if title_details else None
        language = language.parent.parent.find('a') if language else None
        language = language.text if language else None
        movie_details[url]['Language'] = language
        
        primary_filming_location = title_details.find(text=re.compile('Filming Locations:')) if title_details else None
        primary_filming_location = primary_filming_location.parent.parent.find('a') if primary_filming_location else None
        primary_filming_location= primary_filming_location.text if primary_filming_location else None
        movie_details[url]['Primary Filming Location'] = primary_filming_location
        
        budget = title_details.find(text=re.compile('Budget:')) if title_details else None
        budget = budget.parent.parent if budget else None
        budget = budget.text.strip() if budget else None
        budget = re.sub('[^0-9]+','', budget) if budget else None
        movie_details[url]['Budget'] = budget
        
        opening_weekend_usa = title_details.find(text=re.compile('Opening Weekend USA:')) if title_details else None
        opening_weekend_usa = opening_weekend_usa.parent.parent if opening_weekend_usa else None
        opening_weekend_usa = opening_weekend_usa.text.strip() if opening_weekend_usa else None
        opening_weekend_usa = re.sub('[^0-9]+','', opening_weekend_usa) if opening_weekend_usa else None
        movie_details[url]['Opening Weekend USA'] = opening_weekend_usa
        
        #rename gross to gross USA in listing scrape
        global_gross = title_details.find(text=re.compile('Cumulative Worldwide Gross:')) if title_details else None
        global_gross = global_gross.parent.parent if global_gross else None
        global_gross = global_gross.text.strip() if global_gross else None
        global_gross = re.sub('[^0-9]+','', global_gross) if global_gross else None
        movie_details[url]['Cumulative Worldwide Gross'] = global_gross
    
        gross_usa = title_details.find(text=re.compile('Gross USA:')) if title_details else None
        gross_usa = gross_usa.parent.parent.text.strip() if global_gross else None
        gross_usa = re.sub('[^0-9]+','', gross_usa) if gross_usa else None
        movie_details[url]['Gross USA'] = gross_usa
        
        go_to_next_and_export()
    
    
    def go_to_next_and_export():
        global movies_scraped
        global movie_index
        global movie_details
        
        movies_scraped+=1
        
        if len(movie_details) == 2000:                                         
            export_to_json(movie_details)
            movies_scraped = 0
            movie_details = {}
#             return
        
        movie_index += 1
        if movie_index <= 106905:
            loop_through_movies(movie_urls)
        else: 
            export_to_json(movie_details)
            print("Done")      
    

    def export_to_json(movie_details):
        global export_no
        global export_number
        with open('movie_details_export_%s.json' %(export_no),'w')  as f:
            json.dump(movie_details,f)
            
        export_number += 1
        export_no = str(export_number)
        print('Working on export ' + export_no)

    loop_through_movies(movie_urls)

movie_scraper()

Working on export 1


RecursionError: maximum recursion depth exceeded

In [510]:
movies_scraped

981